In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

pd.set_option('display.max_columns', None)

In [3]:
url = 'https://tanki.su/ru/tankopedia/#wot&w_m=tanks&w_n=usa_ussr_germany_china_france_uk_japan_czech_sweden_poland_italy'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

links = [element.get_attribute('href') for element in driver.find_elements(By.CLASS_NAME, 'table-view_row')]

driver.quit()

C:\Users\n.samikaev\AppData\Local\Temp\ipykernel_9692\269499518.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [24]:
driver = webdriver.Chrome('chromedriver.exe')

data = {}

for link in links:
    #  page
    driver.get(link)
    
    # target data holder
    tank_info = {}
    
    # select modules
    for i in range(len(driver.find_elements(By.CLASS_NAME, 'modules_item-wrapper'))):
        driver.find_elements(By.CLASS_NAME, 'modules_item-wrapper')[i].click()
    
    # nation, level and tank_type
    nation, level, tank_type = [x.text for x in driver.find_elements(By.CLASS_NAME, 'tank-statistic_text')[:3]]
    tank_info['Нация'] = nation
    tank_info['Уровень'] = level
    tank_info['Тип танка'] = tank_type
    
    # specification
    for i in range(len(driver.find_elements(By.CLASS_NAME, 'specification_item'))):
        specification = driver.find_elements(By.CLASS_NAME, 'specification_item')[i].text.split('\n')
        try:
            tank_info[specification[1]] = specification[0]
        except:
            continue
            
    # adding tank_info to the dict
    tank_name = driver.find_element(By.XPATH, "//div[@class='garage_title-wrapper']/h1").text
    data[tank_name] = tank_info
    
driver.quit()

C:\Users\n.samikaev\AppData\Local\Temp\ipykernel_9692\274982509.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [38]:
df = pd.DataFrame.from_dict(data)
df = df.transpose()
df = df.reset_index()
df = df.rename({'index': 'Наименование'}, axis=1)
df

,Наименование,Нация,Уровень,Тип танка,Урон,Бронепробиваемость,Скорострельность,Урон в минуту,Время сведения,Разброс на 100 м,...,Скорость гориз. наведения,Мин. время оглушения,Макс. время оглушения,Переход в осадный режим,Переход в походный режим,Угол поворота,Переход в скоростной режим,Переход в обычный режим,Смена режима работы двигателя,Максимальное ускорение перезарядки
0,МС-1,СССР,I,ЛЁГКИЙ ТАНК,47 / 47 / 62 ед.,51 / 88 / 23 мм,"20,69 выстр/мин",972 ед/мин,"2,50 с","0,46 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,БТ-2,СССР,II,ЛЁГКИЙ ТАНК,47 / 47 / 62 ед.,51 / 88 / 23 мм,"26,09 выстр/мин",1 226 ед/мин,"2,30 с","0,44 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Т-26,СССР,II,ЛЁГКИЙ ТАНК,40 / 40 / 50 ед.,58 / 92 / 19 мм,"26,09 выстр/мин",1 043 ед/мин,"2,30 с","0,39 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Т-60,СССР,II,ЛЁГКИЙ ТАНК,40 / 40 / 50 ед.,58 / 92 / 19 мм,"22,22 выстр/мин",888 ед/мин,"2,50 с","0,39 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Т-45,СССР,II,ЛЁГКИЙ ТАНК,47 / 47 / 62 ед.,51 / 84 / 23 мм,"16,67 выстр/мин",783 ед/мин,"2,40 с","0,40 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,SMV CC-56,ИТАЛИЯ,VII,ПТ-САУ,320 / 320 / 420 ед.,175 / 212 / 53 мм,"5,62 выстр/мин",1 799 ед/мин,"2,70 с","0,42 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,SMV CC-67,ИТАЛИЯ,VIII,ПТ-САУ,320 / 320 / 420 ед.,220 / 242 / 105 мм,"6,43 выстр/мин",2 057 ед/мин,"2,20 с","0,36 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,SMV CC-64 VIPERA,ИТАЛИЯ,VIII,ПТ-САУ,400 / 400 / 515 ед.,220 / 264 / 120 мм,5 выстр/мин,2 000 ед/мин,3 с,"0,44 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,CONTROCARRO 1 MK. 2,ИТАЛИЯ,IX,ПТ-САУ,490 / 490 / 640 ед.,250 / 325 / 127 мм,5 выстр/мин,2 450 ед/мин,"2,70 с","0,42 м",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df.to_excel('Techincs.xlsx', index=False)